# What Should I Serve?
## A Study of Restaurants in New Orleans Neighborhoods

#### Completed in partial completion of IBM's Data Science Professional Certification - Tony Allen Price

### Introduction

Imagine for a moment that you are an restaurantuer looking for new opportunties in the city of New Orleans, Louisiana. You know that the city is famous the world over for the lively food scene, and you cannot imagine opening your new venture anywhere else. There is a downside to going to such a popular food desitnation, however. There are already hundreds, if not thousands, of thriving restaurants competing for local and tourist dollars alike. YOu know that you have to find some way to stand out in the crowd. How can you find that 'x factor' to make your new restaurant different from all the others?

Data Science, of course! You decide that that what you need to do is get a deeper look at the restaurants currently serving the diverse and liveley neighborhoods in New Orleans. Each area of the city is home to restaurants serving diffrent menus to different clientle. In order to stand out from the crowd, you need to know what each neighborhood contains, and what each neighborhood lacks. Your plan is to find an area the Big Easy that isn't too crowded, but still has thriving businesses, and then to open up a new eatery that is serving something unlike all the others. 

**Key Questions:**

1. How many restaraunts are in each of the neighborhoods in New Orleans?
2. What type of food do those restraunts serve?
3. Where is there room in the market for something new?
4. What would be a unique food to serve in that neighborhood?

### About the Data

Based on the questions we are trying to answer, we need to collect data on a few different topics. First, we need to know more about the neighborhoods of New Orleans. Luckily, Wikipedia has a very helpful table of [all the neighborhoods and thier geographic coordinates](https://en.wikipedia.org/wiki/Neighborhoods_in_New_Orleans) that we can use.

Next, we need to be able to identify the number and type of restraunts in a given neighborhood. We will use the API of FourSquare to help us find and identify this information. 

Finally, we will combine this information and perform analysis and visualization techniques using *pandas*, *numpy*, *folium* and other packages to decide how densely packed a neighborhood is with restaurants as well as to idenify the types of restaurants that are located in each neighborhood.

In the end, we should be able to develop a simple search and retrieve function that allows you to input a neighborhood name, and get back an interactive map with the information you are looking for.